# Energy System Simulation Analysis

This notebook analyzes the results from our off-grid energy management system simulation, focusing on three key aspects:
1. Battery State of Charge (SOC)
2. Predicted Solar Production
3. Predicted Load Consumption

We'll create visualizations to understand the relationships between these components and evaluate system performance.

In [15]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Load simulation results
df = pd.read_csv('../results/phase_2_offgrid_simulation.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set timestamp as index for time series analysis
df.set_index('timestamp', inplace=True)
print("Data loaded successfully:")
print(f"Period: {df.index.min()} to {df.index.max()}")
print(f"Total hours: {len(df):,}")
print(f"Total days: {len(df)/24:.1f}")

Data loaded successfully:
Period: 2024-10-01 00:00:00 to 2024-12-31 00:00:00
Total hours: 2,185
Total days: 91.0


# Combined System Performance Visualization

Let's create a comprehensive visualization that shows the relationship between:
- Battery State of Charge (%)
- Predicted Solar Production (kW)
- Predicted Load Consumption (kW)

This will help us understand how these components interact and affect system performance.

In [17]:
# Create the combined visualization
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add battery SOC
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['battery_soc_percent'],
        name="Battery SOC (%)",
        line=dict(color='blue', width=2),
        hovertemplate="SOC: %{y:.1f}%<br>Time: %{x}",
    ),
    secondary_y=True,
)

# Add predicted production
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['predicted_production_kw'],
        name="Solar Production (kW)",
        line=dict(color='orange', width=2),
        hovertemplate="Production: %{y:.2f} kW<br>Time: %{x}",
    ),
    secondary_y=False,
)

# Add predicted consumption
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['predicted_consumption_kw'],
        name="Load Consumption (kW)",
        line=dict(color='red', width=2),
        hovertemplate="Consumption: %{y:.2f} kW<br>Time: %{x}",
    ),
    secondary_y=False,
)

# Add battery operating range indicators
fig.add_hline(y=80, line=dict(color="rgba(0,0,255,0.3)", width=1, dash="dash"), secondary_y=True)
fig.add_hline(y=20, line=dict(color="rgba(0,0,255,0.3)", width=1, dash="dash"), secondary_y=True)

# Update layout
fig.update_layout(
    title="Energy System Performance Overview",
    height=800,
    hovermode="x unified",
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

# Update axes titles
fig.update_yaxes(title_text="Power (kW)", secondary_y=False)
fig.update_yaxes(title_text="Battery SOC (%)", secondary_y=True)
fig.update_xaxes(title_text="Time")

# Show plot
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# Daily Patterns Analysis

Let's analyze the average daily patterns of production and consumption, along with how the battery SOC typically evolves throughout the day. This will help us understand:
- Peak production hours
- Peak consumption periods
- Typical battery behavior
- System stress points

In [ ]:
# Calculate average patterns by hour
hourly_patterns = df.groupby(df.index.hour).agg({
    'battery_soc_percent': 'mean',
    'predicted_production_kw': 'mean',
    'predicted_consumption_kw': 'mean'
}).round(2)

# Create daily patterns plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add battery SOC pattern
fig.add_trace(
    go.Scatter(
        x=hourly_patterns.index,
        y=hourly_patterns['battery_soc_percent'],
        name="Avg Battery SOC (%)",
        line=dict(color='blue', width=2),
        hovertemplate="Hour: %{x}<br>Avg SOC: %{y:.1f}%",
    ),
    secondary_y=True,
)

# Add production pattern
fig.add_trace(
    go.Scatter(
        x=hourly_patterns.index,
        y=hourly_patterns['predicted_production_kw'],
        name="Avg Production (kW)",
        line=dict(color='orange', width=2),
        hovertemplate="Hour: %{x}<br>Avg Production: %{y:.2f} kW",
    ),
    secondary_y=False,
)

# Add consumption pattern
fig.add_trace(
    go.Scatter(
        x=hourly_patterns.index,
        y=hourly_patterns['predicted_consumption_kw'],
        name="Avg Consumption (kW)",
        line=dict(color='red', width=2),
        hovertemplate="Hour: %{x}<br>Avg Consumption: %{y:.2f} kW",
    ),
    secondary_y=False,
)

# Update layout
fig.update_layout(
    title="Average Daily Patterns",
    height=600,
    hovermode="x unified",
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)"
    ),
    xaxis=dict(
        title="Hour of Day",
        tickmode='array',
        ticktext=[f"{i:02d}:00" for i in range(24)],
        tickvals=list(range(24))
    )
)

# Update axes titles
fig.update_yaxes(title_text="Power (kW)", secondary_y=False)
fig.update_yaxes(title_text="Battery SOC (%)", secondary_y=True)

# Show plot
fig.show()

# Print summary statistics
print("\nSummary Statistics:")
print("=" * 50)
print("\nPeak Hours:")
print(f"Production: {hourly_patterns['predicted_production_kw'].idxmax():02d}:00 ({hourly_patterns['predicted_production_kw'].max():.2f} kW)")
print(f"Consumption: {hourly_patterns['predicted_consumption_kw'].idxmax():02d}:00 ({hourly_patterns['predicted_consumption_kw'].max():.2f} kW)")
print(f"\nBattery SOC Range:")
print(f"Maximum: {hourly_patterns['battery_soc_percent'].max():.1f}%")
print(f"Minimum: {hourly_patterns['battery_soc_percent'].min():.1f}%")

# System Performance Metrics

Let's calculate and visualize key performance metrics:
1. Energy Balance (Production vs. Consumption)
2. Battery Utilization
3. System Reliability
4. Blackout Analysis

In [ ]:
# Calculate system performance metrics
total_production = df['predicted_production_kw'].sum()
total_consumption = df['predicted_consumption_kw'].sum()
total_blackout_hours = df['is_blackout'].sum()
total_hours = len(df)

metrics = {
    'Total Production (kWh)': total_production,
    'Total Consumption (kWh)': total_consumption,
    'Energy Balance (kWh)': total_production - total_consumption,
    'Self-Sufficiency Ratio (%)': (total_production / total_consumption * 100) if total_consumption > 0 else 0,
    'Average Battery SOC (%)': df['battery_soc_percent'].mean(),
    'System Reliability (%)': ((total_hours - total_blackout_hours) / total_hours * 100),
    'Total Blackout Hours': total_blackout_hours,
    'Blackout Rate (%)': (total_blackout_hours / total_hours * 100)
}

# Create metrics visualization
fig = go.Figure()

# Add metrics as a table
fig.add_trace(go.Table(
    header=dict(
        values=['Metric', 'Value'],
        fill_color='royalblue',
        align='left',
        font=dict(color='white', size=12)
    ),
    cells=dict(
        values=[
            list(metrics.keys()),
            [f"{v:.2f}" if isinstance(v, float) else v for v in metrics.values()]
        ],
        align='left',
        fill_color='lavender'
    )
))

fig.update_layout(
    title="System Performance Summary",
    height=400,
    width=800
)

fig.show()

# Create pie chart for energy balance
fig = go.Figure(data=[
    go.Pie(
        labels=['Production', 'Consumption', 'Surplus/Deficit'],
        values=[total_production, total_consumption, abs(total_production - total_consumption)],
        hole=.3,
        hovertemplate="%{label}<br>%{value:.1f} kWh<br>%{percent:.1%}<extra></extra>"
    )
])

fig.update_layout(
    title="Energy Balance Distribution",
    height=500,
    width=700
)

fig.show()